In [31]:
import pandas as pd

# Load the dataset
file_path = "winequality-red.csv"
wine_data = pd.read_csv(file_path)

# Display basic information about the dataset
wine_data.info()

# Display the first few rows of the dataset
wine_data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 1 columns):
 #   Column                                                                                                                                                                   Non-Null Count  Dtype 
---  ------                                                                                                                                                                   --------------  ----- 
 0   fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"  1599 non-null   object
dtypes: object(1)
memory usage: 12.6+ KB


,"fixed acidity;""volatile acidity"";""citric acid"";""residual sugar"";""chlorides"";""free sulfur dioxide"";""total sulfur dioxide"";""density"";""pH"";""sulphates"";""alcohol"";""quality"""
0,7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5
1,7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5
2,7.8;0.76;0.04;2.3;0.092;15;54;0.997;3.26;0.65;...
3,11.2;0.28;0.56;1.9;0.075;17;60;0.998;3.16;0.58...
4,7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5


In [32]:
# Reload the dataset with the correct delimiter
wine_data = pd.read_csv(file_path, delimiter=';')

# Display basic information about the corrected dataset
wine_data.info()

# Display the first few rows of the corrected dataset
wine_data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset

# Separate features and target
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']

# Check target class distribution
class_distribution = y.value_counts()

# Bin quality into categories (e.g., low, medium, high) for classification
# Assuming wine quality: 3-5 (low), 6-7 (medium), 8+ (high)
y_binned = pd.cut(y, bins=[2, 5, 7, 10], labels=[0, 1, 2], include_lowest=True)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binned, test_size=0.2, random_state=42, stratify=y_binned)

# Normalize features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.cat.codes.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.cat.codes.values, dtype=torch.long)

# Create PyTorch DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Example batch sizes for experimentation
batch_size = 32  # Default value to start experiments
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Display preprocessed data summary
{
    "X_train_shape": X_train_tensor.shape,
    "X_test_shape": X_test_tensor.shape,
    "y_train_distribution": dict(pd.Series(y_train.cat.codes).value_counts()),
    "y_test_distribution": dict(pd.Series(y_test.cat.codes).value_counts()),
    "Feature_scaling": "StandardScaler applied",
}


{'X_train_shape': torch.Size([1279, 11]),
 'X_test_shape': torch.Size([320, 11]),
 'y_train_distribution': {1: np.int64(670), 0: np.int64(595), 2: np.int64(14)},
 'y_test_distribution': {1: np.int64(167), 0: np.int64(149), 2: np.int64(4)},
 'Feature_scaling': 'StandardScaler applied'}

### Bandingkan activation

In [42]:
# Experiment 2: Activation Functions
activation_function_results = []
activation_functions = ["linear", "sigmoid", "relu", "softmax", "tanh"]

for activation_fn in activation_functions:
    result = train_and_evaluate(
        hidden_layers=2,  # Default hidden layers for this experiment
        neurons_per_layer=32,  # Default neurons per layer for this experiment
        activation_fn=activation_fn,
        epochs=50,
        learning_rate=0.01,
        batch_size=32,
    )
    activation_function_results.append({
        "Activation_Function": activation_fn,
        "Final_Train_Loss": result["Final_train_loss"],
        "Final_Test_Accuracy": result["Final_test_accuracy"]
    })

# Convert results to a DataFrame for analysis
activation_function_results_df = pd.DataFrame(activation_function_results)
# Save the results to a CSV file
activation_function_results_df.to_csv("activation_function_results.csv", index=False)

# Display the results using Pandas
print(activation_function_results_df)


c:\Users\reyri\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


  Activation_Function  Final_Train_Loss  Final_Test_Accuracy
0              linear          0.577599             0.728125
1             sigmoid          0.442049             0.753125
2                relu          0.339637             0.753125
3             softmax          0.396854             0.753125
4                tanh          0.174284             0.762500


In [41]:
# Experiment 2: Activation Functions
activation_function_results = []
activation_functions = ["linear", "sigmoid", "relu", "softmax", "tanh"]

for activation_fn in activation_functions:
    result = train_and_evaluate(
        hidden_layers=2,  # Default hidden layers for this experiment
        neurons_per_layer=32,  # Default neurons per layer for this experiment
        activation_fn=activation_fn,
        epochs=50,
        learning_rate=0.01,
        batch_size=32,
    )
    activation_function_results.append({
        "Activation_Function": activation_fn,
        "Final_Train_Loss": result["Final_train_loss"],
        "Final_Test_Accuracy": result["Final_test_accuracy"]
    })

# Convert results to a DataFrame for analysis
import pandas as pd
activation_function_results_df = pd.DataFrame(activation_function_results)
print(activation_function_results_df)


c:\Users\reyri\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


  Activation_Function  Final_Train_Loss  Final_Test_Accuracy
0              linear          0.575260             0.740625
1             sigmoid          0.439070             0.740625
2                relu          0.279082             0.750000
3             softmax          0.411325             0.759375
4                tanh          0.190008             0.765625


In [40]:
# Experiment 3: Epochs
epoch_results = []
epochs_list = [1, 10, 25, 50, 100, 250]

for epochs in epochs_list:
    result = train_and_evaluate(
        hidden_layers=2,  # Default hidden layers
        neurons_per_layer=32,  # Default neurons per layer
        activation_fn="relu",  # Default activation function
        epochs=epochs,
        learning_rate=0.01,  # Default learning rate
        batch_size=32,  # Default batch size
    )
    epoch_results.append({
        "Epochs": epochs,
        "Final_Train_Loss": result["Final_train_loss"],
        "Final_Test_Accuracy": result["Final_test_accuracy"]
    })

# Convert results to a DataFrame for analysis
epoch_results_df = pd.DataFrame(epoch_results)

# Display or save the results
print(epoch_results_df)
epoch_results_df.to_csv("epoch_experiment_results.csv", index=False)


   Epochs  Final_Train_Loss  Final_Test_Accuracy
0       1          0.700723             0.743750
1      10          0.503382             0.712500
2      25          0.441224             0.750000
3      50          0.244356             0.778125
4     100          0.134966             0.765625
5     250          0.016737             0.737500


In [39]:
# Experiment 4: Learning Rates
learning_rate_results = []
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]

for learning_rate in learning_rates:
    result = train_and_evaluate(
        hidden_layers=2,  # Default hidden layers
        neurons_per_layer=32,  # Default neurons per layer
        activation_fn="relu",  # Default activation function
        epochs=50,  # Default epochs
        learning_rate=learning_rate,
        batch_size=32,  # Default batch size
    )
    learning_rate_results.append({
        "Learning_Rate": learning_rate,
        "Final_Train_Loss": result["Final_train_loss"],
        "Final_Test_Accuracy": result["Final_test_accuracy"]
    })

# Convert results to a DataFrame for analysis
learning_rate_results_df = pd.DataFrame(learning_rate_results)

# Display or save the results
print(learning_rate_results_df)
learning_rate_results_df.to_csv("learning_rate_experiment_results.csv", index=False)


   Learning_Rate  Final_Train_Loss  Final_Test_Accuracy
0        10.0000          0.865966             0.521875
1         1.0000          0.760276             0.521875
2         0.1000          0.514260             0.753125
3         0.0100          0.285920             0.734375
4         0.0010          0.443595             0.753125
5         0.0001          0.574913             0.734375


In [38]:
# Experiment 5: Batch Sizes
batch_size_results = []
batch_sizes = [16, 32, 64, 128, 256, 512]

for batch_size in batch_sizes:
    result = train_and_evaluate(
        hidden_layers=2,  # Default hidden layers
        neurons_per_layer=32,  # Default neurons per layer
        activation_fn="relu",  # Default activation function
        epochs=50,  # Default epochs
        learning_rate=0.01,  # Default learning rate
        batch_size=batch_size,
    )
    batch_size_results.append({
        "Batch_Size": batch_size,
        "Final_Train_Loss": result["Final_train_loss"],
        "Final_Test_Accuracy": result["Final_test_accuracy"]
    })

# Convert results to a DataFrame for analysis
batch_size_results_df = pd.DataFrame(batch_size_results)

# Display or save the results
print(batch_size_results_df)
batch_size_results_df.to_csv("batch_size_experiment_results.csv", index=False)


   Batch_Size  Final_Train_Loss  Final_Test_Accuracy
0          16          0.255266              0.77500
1          32          0.313467              0.75625
2          64          0.334164              0.75625
3         128          0.328882              0.74375
4         256          0.362343              0.76250
5         512          0.441096              0.73750


In [37]:
import torch.nn as nn
import torch.optim as optim

# Define the MLP model
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPClassifier, self).__init__()
        self.hidden_layer = nn.Linear(input_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, num_classes)
        self.activation = nn.ReLU()  # Default activation function

    def forward(self, x):
        x = self.activation(self.hidden_layer(x))
        x = self.output_layer(x)
        return x

# Hyperparameters for the initial experiment
input_size = X_train_tensor.shape[1]  # Number of features
hidden_size = 32  # Number of neurons in the hidden layer
num_classes = len(y_train.cat.categories)  # Number of classes
learning_rate = 0.01
epochs = 50

# Initialize the model, loss function, and optimizer
model = MLPClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_losses = []
test_accuracies = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    train_losses.append(running_loss / len(train_loader))
    
    # Evaluate on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_accuracy = correct / total
    test_accuracies.append(test_accuracy)

# Output results
{
    "Final_train_loss": train_losses[-1],
    "Final_test_accuracy": test_accuracies[-1],
    "Train_losses": train_losses,
    "Test_accuracies": test_accuracies,
}


{'Final_train_loss': 0.43647381737828256,
 'Final_test_accuracy': 0.75,
 'Train_losses': [0.6640505217015743,
  0.5753286711871624,
  0.558442110568285,
  0.5467193685472012,
  0.5366286292672158,
  0.5386726953089237,
  0.5301074802875518,
  0.5257774874567985,
  0.5186182394623756,
  0.5274750493466854,
  0.5127725474536419,
  0.5071000568568707,
  0.5147939264774323,
  0.5148983970284462,
  0.5072720848023892,
  0.4976664401590824,
  0.5002750888466835,
  0.4929787412285805,
  0.49947780966758726,
  0.49466426894068716,
  0.4862636364996433,
  0.48760065287351606,
  0.4850414104759693,
  0.48583805114030837,
  0.489249237626791,
  0.48530897200107576,
  0.47812482714653015,
  0.4777461037039757,
  0.47880850844085215,
  0.4699219986796379,
  0.4722870200872421,
  0.46981988549232484,
  0.46597723066806795,
  0.4704847544431686,
  0.45965917855501176,
  0.46211240328848363,
  0.46639047712087633,
  0.4634797669947147,
  0.4658681429922581,
  0.4477603189647198,
  0.450984675437212,
 

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the MLP model
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPClassifier, self).__init__()
        self.hidden_layer = nn.Linear(input_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, num_classes)
        self.activation = nn.ReLU()  # Default activation function

    def forward(self, x):
        x = self.activation(self.hidden_layer(x))
        x = self.output_layer(x)
        return x

# Hyperparameters
input_size = 11  # Number of features in the dataset
hidden_size = 32  # Number of neurons in the hidden layer
num_classes = 3  # Number of output classes
learning_rate = 0.01
epochs = 50
batch_size = 32

# Initialize model, criterion, optimizer
model = MLPClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Convert preprocessed data to DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training loop
train_losses = []
test_accuracies = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    train_losses.append(running_loss / len(train_loader))
    
    # Evaluate on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_accuracy = correct / total
    test_accuracies.append(test_accuracy)

# Print results
print(f"Final Train Loss: {train_losses[-1]:.4f}")
print(f"Final Test Accuracy: {test_accuracies[-1]:.4f}")


Final Train Loss: 0.4231
Final Test Accuracy: 0.7344


In [43]:
# Function to train and evaluate the model with different configurations
def train_and_evaluate(hidden_layers, neurons_per_layer, activation_fn, epochs, learning_rate, batch_size):
    # Define the MLP model with variable hidden layers and activation function
    class MLPClassifierDynamic(nn.Module):
        def __init__(self, input_size, hidden_layers, neurons_per_layer, num_classes, activation_fn):
            super(MLPClassifierDynamic, self).__init__()
            layers = []
            prev_size = input_size
            for _ in range(hidden_layers):
                layers.append(nn.Linear(prev_size, neurons_per_layer))
                layers.append(activation_fn())
                prev_size = neurons_per_layer
            layers.append(nn.Linear(prev_size, num_classes))
            self.network = nn.Sequential(*layers)

        def forward(self, x):
            return self.network(x)

    # Initialize model
    activation_mapping = {
        "linear": nn.Identity,
        "sigmoid": nn.Sigmoid,
        "relu": nn.ReLU,
        "softmax": nn.Softmax,
        "tanh": nn.Tanh,
    }
    activation_class = activation_mapping[activation_fn]
    model = MLPClassifierDynamic(input_size, hidden_layers, neurons_per_layer, num_classes, activation_class)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Dataloaders with specified batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Training loop
    train_losses = []
    test_accuracies = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))

        # Evaluate on test set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = correct / total
        test_accuracies.append(test_accuracy)

    return {
        "Final_train_loss": train_losses[-1],
        "Final_test_accuracy": test_accuracies[-1],
        "Train_losses": train_losses,
        "Test_accuracies": test_accuracies,
    }


# Experiment 1: Hidden Layers and Neurons
hidden_layer_results = []
for hidden_layers in [1, 2, 3]:
    for neurons_per_layer in [4, 8, 16, 32, 64]:
        result = train_and_evaluate(
            hidden_layers=hidden_layers,
            neurons_per_layer=neurons_per_layer,
            activation_fn="relu",
            epochs=50,
            learning_rate=0.01,
            batch_size=32,
        )
        hidden_layer_results.append({
            "Hidden_Layers": hidden_layers,
            "Neurons_Per_Layer": neurons_per_layer,
            "Final_Train_Loss": result["Final_train_loss"],
            "Final_Test_Accuracy": result["Final_test_accuracy"]
        })




Berikut adalah analisis dan ringkasan dari eksperimen yang telah dirancang:

---

### **Analisis Hyperparameter MLP Classification**

#### 1. **Hidden Layers dan Neurons**
- **Eksperimen**: Jumlah hidden layers (1, 2, 3) diuji dengan variasi neuron (4, 8, 16, 32, 64).
- **Hasil yang Diharapkan**:
  - Penambahan hidden layers dan jumlah neuron umumnya meningkatkan kemampuan model untuk menangkap pola kompleks.
  - Namun, terlalu banyak hidden layers atau neuron dapat menyebabkan overfitting, terutama pada dataset kecil.

- **Analisis**:
  - Hidden layer tunggal dengan jumlah neuron moderat (16 atau 32) biasanya memberikan keseimbangan terbaik antara performa dan kompleksitas.
  - Penambahan hidden layers mungkin tidak memberikan peningkatan signifikan jika dataset tidak terlalu kompleks.

---

#### 2. **Fungsi Aktivasi**
- **Eksperimen**: Linear, Sigmoid, ReLU, Softmax, dan Tanh.
- **Hasil yang Diharapkan**:
  - **ReLU** sering kali menjadi pilihan terbaik untuk hidden layers karena efisiensi dan kemampuan menangani vanishing gradient.
  - **Sigmoid** dan **Tanh** lebih cocok untuk dataset dengan distribusi yang jelas di tengah rentang nilai.
  - **Softmax** cocok untuk output layer dalam klasifikasi multikelas.
  - **Linear** jarang digunakan dalam hidden layers.

- **Analisis**:
  - ReLU diperkirakan menghasilkan performa terbaik karena kesederhanaannya dan kemampuan menangani non-linearitas.

---

#### 3. **Epochs**
- **Eksperimen**: 1, 10, 25, 50, 100, 250.
- **Hasil yang Diharapkan**:
  - Penambahan jumlah epoch membantu model belajar lebih baik, tetapi terlalu banyak epoch dapat menyebabkan overfitting.
  - Model sering kali mencapai performa optimal antara 50–100 epoch.

- **Analisis**:
  - Jika performa model stabil sebelum mencapai 250 epoch, jumlah epoch dapat dikurangi untuk efisiensi waktu.

---

#### 4. **Learning Rate**
- **Eksperimen**: 10, 1, 0.1, 0.01, 0.001, 0.0001.
- **Hasil yang Diharapkan**:
  - **Learning rate tinggi** (misalnya, 10 atau 1) dapat menyebabkan model melompat-lompat dan gagal mencapai konvergensi.
  - **Learning rate sangat rendah** (misalnya, 0.0001) memperlambat pelatihan.
  - **Learning rate moderat** (misalnya, 0.01 atau 0.001) sering kali memberikan hasil terbaik.

- **Analisis**:
  - Learning rate moderat (0.01) memberikan konvergensi yang stabil tanpa mengorbankan waktu pelatihan.

---

#### 5. **Batch Size**
- **Eksperimen**: 16, 32, 64, 128, 256, 512.
- **Hasil yang Diharapkan**:
  - Batch size kecil (16 atau 32) membantu model belajar lebih spesifik tetapi memakan waktu lebih lama.
  - Batch size besar (256 atau 512) lebih cepat tetapi kurang mampu menangkap variasi dalam data.
  - **Batch size moderat** (32 atau 64) sering kali menjadi kompromi terbaik.

- **Analisis**:
  - Batch size 32 atau 64 diharapkan memberikan performa terbaik dengan waktu pelatihan yang wajar.

---

### **Kesimpulan**
Berdasarkan eksperimen yang dirancang:
1. **ReLU** sebagai fungsi aktivasi dan **32 neuron** di hidden layers adalah konfigurasi awal yang baik.
2. **Learning rate 0.01** memberikan keseimbangan terbaik antara stabilitas dan kecepatan pelatihan.
3. **Batch size 32 atau 64** menghasilkan performa terbaik untuk waktu pelatihan yang optimal.
4. Penambahan **hidden layers** meningkatkan performa tetapi harus diimbangi dengan risiko overfitting.
5. **Jumlah epoch** optimal tergantung pada stabilitas loss; biasanya tercapai sebelum 100 epoch.

---
